---
# Carregar Dataset
---

In [ ]:
import numpy as np
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.ensemble import RandomForestClassifier
import statistics
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, KFold, train_test_split,StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
#from tensorflow.keras.models  import Sequential
#from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

RANDOM_SEED = 2022

In [ ]:
df = pd.read_csv("training_data.csv")
df_test = pd.read_csv("test_data.csv")

---
# Exploração de dados
---

- **city_name** - nome da cidade em causa;
- **record_date** - o timestamp associado ao registo;
- **magnitude_of_delay** - magnitude do atraso provocado pelos incidentes que se verificam no record_date correspondente;
- **delay_in_seconds** - atraso, em segundos, provocado pelos incidentes que se verificam no record_date correspondente;
- **affected_roads** - estradas afectadas pelos incidentes que se verificam no record_date correspondente;
- **luminosity** - o nível de luminosidade que se verificava na cidade de Guimarães;
- **avg_temperature** - valor médio da temperatura para o record_date na cidade de Guimarães;
- **avg_atm_pressure** - valor médio da pressão atmosférica para o record_date na cidade de Guimarães;
- **avg_humidity** - valor médio de humidade para o record_date na cidade de Guimarães;
- **avg_wind_speed** - valor médio da velocidade do vento para o record_date na cidade de Guimarães;
- **avg_precipitation** - valor médio de precipitação para o record_date na cidade de Guimarães;
- **avg_rain** - avaliação qualitativa do nível de precipitação para o record_date na cidade de Guimarães;
- **incidents** - indicação acerca do nível de incidentes rodoviários que se verificam no record_date correspondente na cidade de Guimarães.

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
print("------------\n" + "Value counts (incidents):\n\n" + df['incidents'].value_counts().to_string() + "\n------------\n")

for s in ['city_name','magnitude_of_delay','luminosity','avg_rain']:
    print("------------\n" + "Value counts (" + s + "):\n\n" + "training dataset:\n" + df[s].value_counts().to_string() + 
          "\n\ntest dataset:\n" + df_test[s].value_counts().to_string() + "\n------------\n")

In [ ]:
print("training dataset:" ,df['avg_precipitation'].value_counts())
print("\ntest dataset:" ,df['avg_precipitation'].value_counts())

In [ ]:
#Values are always the same (irrelevant features)
df.drop(['city_name','avg_precipitation'], axis=1, inplace=True)
df_test.drop(['city_name','avg_precipitation'], axis=1, inplace=True)

In [ ]:
df.describe()

In [ ]:
print(f"Skewness: \n{df.skew()}")
print(f"\nKurtosis: \n{df.kurt()}")

In [ ]:
sns.pairplot(df)

In [ ]:
sns.catplot(y='delay_in_seconds', x='magnitude_of_delay', data=df, kind='box')

In [ ]:
corr_matrix = df.corr()
sns.heatmap(corr_matrix, vmin=-1, vmax=1, square=True, annot=True)
#Strong correlation between the features avg_temperature and avg_humidity

In [ ]:
#Only the feature affected_roads has missing values
df.isna().sum()

----------
* Tratamento das roads

In [ ]:
#Function to remove duplicate roads
import re
def updateRoadSet(val,setRoads):
    pattern = re.compile(",?(\w[ \w\-]*\w|\w),?")
    lista = re.findall(pattern,val)
    
    for road in lista:
        setRoads.add(road)

In [ ]:
# Remover roads duplicadas
setRoads = set()
df['affected_roads'].fillna(",").apply(updateRoadSet,args=(setRoads,))

print(setRoads)

In [ ]:
for road in setRoads:
    df[road] = 0
df

In [ ]:
import re
def get_roads(roads_string : str):
    pattern = re.compile(",?(\w[ \w\-]*\w|\w),?")
    l = re.findall(pattern,roads_string)
    roads = dict()
    for r in l:
        road_count = roads.get(r)
        if(road_count != None):
            roads[r] = road_count + 1
        else:
            roads[r] = 1
    return roads

In [ ]:
df['affected_roadsString'] = df['affected_roads'].astype('string')

In [ ]:
for i in range (0, df.shape[0]):
    row = df.iloc[i]
    roads_string = str(row['affected_roads'])
    roads = get_roads(roads_string)
    for road,counter in roads.items():
        df.at[i, road] = counter
    

In [ ]:
df.loc[math.isnan(float(df['nan']))]

In [ ]:
groupByIncidents = df.groupby("incidents")
groupByIncidents.first()

In [ ]:
groupByIncidents.get_group('Low')

----------
* Tratamento dos duplicados

In [ ]:
#No duplicated entries
df.duplicated().value_counts()

---
# Tratamento dos dados
---

In [ ]:
#'affected roads' separar as entradas em várias por cada estrada
df.drop(['affected_roads'], axis=1, inplace=True)
df_test.drop(['affected_roads'], axis=1, inplace=True)

In [ ]:
#'record_date' separar em diversos campos (horas(??),dia,mes,trimestre,semestre,ano)
#combinar dia da semana com epocas??

df['record_date'] = pd.to_datetime(df['record_date'])
df_test['record_date'] = pd.to_datetime(df_test['record_date'])
#df['hour'] = df['record_date'].dt.hour
#df['day'] = df['record_date'].dt.day
df['day_of_week'] = df['record_date'].dt.dayofweek
df_test['day_of_week'] = df_test['record_date'].dt.dayofweek
#df['week'] = df['record_date'].dt.week
#df['month'] = df['record_date'].dt.month
#df['year'] = df['record_date'].dt.year  # Removed because all occurrences happen in 2021.
df.drop(['record_date'], axis=1, inplace=True)
df_test.drop(['record_date'], axis=1, inplace=True)


In [ ]:
def magnitude_of_delay_label_encoder(magnitude_of_delay):
    if magnitude_of_delay == 'UNDEFINED':
        return 0;
    elif magnitude_of_delay == 'MODERATE':
        return 1;
    elif magnitude_of_delay == 'MAJOR':
        return 2;
    else:
        return -1;
    
df['magnitude_of_delay_cat'] = df['magnitude_of_delay'].apply(magnitude_of_delay_label_encoder)
df_test['magnitude_of_delay_cat'] = df_test['magnitude_of_delay'].apply(magnitude_of_delay_label_encoder) 
df.drop(['magnitude_of_delay'], axis=1, inplace=True)
df_test.drop(['magnitude_of_delay'], axis=1, inplace=True) 

In [ ]:
def luminosity_label_encoder(luminosity):
    if luminosity == 'DARK':
        return 0;
    elif luminosity == 'LOW_LIGHT':
        return 1;
    elif luminosity == 'LIGHT':
        return 2;
    else:
        return -1;
    
df['luminosity_cat'] = df['luminosity'].apply(luminosity_label_encoder)
df_test['luminosity_cat'] = df_test['luminosity'].apply(luminosity_label_encoder) 
df.drop(['luminosity'], axis=1, inplace=True)
df_test.drop(['luminosity'], axis=1, inplace=True) 

In [ ]:
def avg_rain_label_encoder(avg_rain):
    if avg_rain == 'Sem Chuva':
        return 0;
    elif avg_rain == 'chuva fraca':
        return 1;
    elif avg_rain == 'chuva moderada':
        return 2;
    elif avg_rain == 'chuva forte':
        return 3;
    else:
        return -1;
    
df['avg_rain_cat'] = df['avg_rain'].apply(avg_rain_label_encoder)
df_test['avg_rain_cat'] = df_test['avg_rain'].apply(avg_rain_label_encoder)
df.drop(['avg_rain'], axis=1, inplace=True)
df_test.drop(['avg_rain'], axis=1, inplace=True)  

In [ ]:
def incidents_label_encoder(incidents):
    if incidents == 'None':
        return 0;
    elif incidents == 'Low':
        return 1;
    elif incidents == 'Medium':
        return 2;
    elif incidents == 'High':
        return 3;
    elif incidents == 'Very_High':
        return 4;
    else:
        return -1;
    
def incidents_label_decoder(incidents_cat):
    if incidents_cat == 0:
        return 'None'
    elif incidents_cat == 1:
        return 'Low';
    elif incidents_cat == 2:
        return 'Medium';
    elif incidents_cat == 3:
        return 'High';
    elif incidents_cat == 4:
        return 'Very_High';
    else:
        return ""
    
df['incidents_cat'] = df['incidents'].apply(incidents_label_encoder)

In [ ]:
#for feature in ['magnitude_of_delay', 'luminosity', 'avg_rain', 'incidents']:
#    print(df[[feature, feature + '_cat']].drop_duplicates().sort_values(by=[feature + '_cat']).reset_index(drop=True),"\n")

In [ ]:
#order columns
df.sort_index(axis=1)

---
# Modelos
---

In [ ]:
x = df.drop(['incidents','incidents_cat'],axis=1)
y = df['incidents_cat'].to_frame()

In [ ]:
x

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier


clf = DecisionTreeClassifier(random_state=2022, criterion='gini', max_depth=10)
scores = cross_val_score(clf, x, y, cv=10)
print(scores)
print('RESULT: %0.2f accuracy with a standard deviation of %0.2f' % (scores.mean(), scores.std()))
print(scores.mean())

In [ ]:
x_train_data = df.drop(['incidents','incidents_cat'],axis=1)
y_train_data = df['incidents_cat'].to_frame()
clf.fit(x_train_data, y_train_data)
predictions = clf.predict(df_test)
for p in predictions:
    print(incidents_label_decoder(p))